# Annotated dataset

Import and merge files from all annotators:

In [1]:
import pandas as pd

df_karina = pd.read_excel("Annotation_Karina.xlsx")
df_yana = pd.read_excel("Annotation_Yana.xlsx")
df_polina = pd.read_excel("Annotation_Polina.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Annotation_Karina.xlsx'

In [ ]:
common_cols = ['dataset','id_a', 'id_q', 'meta.pair_idx', 'text_q', 'text_a']
annotated = pd.merge(df_karina, df_yana, how='left', left_on=common_cols, right_on=common_cols)
annotated = pd.merge(annotated, df_polina, how='left', left_on=common_cols, right_on=common_cols)

annotated.columns = common_cols + ['if_q_1', 'avoid_rate_1', 'avoid_type_1', 
                                  'if_q_2', 'avoid_rate_2', 'avoid_type_2',
                                  'if_q_3', 'avoid_rate_3', 'avoid_type_3']
print(annotated.shape)
annotated.head(1)

Remove non-questions based on majority value, count the average rate and type of avoidance:

In [ ]:
# Filter non-questions
ann = annotated[(annotated[['if_q_1','if_q_2','if_q_3']] == 'NQ').sum(axis=1) < 2]
ann = ann.reset_index()
ann.shape[0]

424

In [ ]:
# Count average rate of avoidance
ann['avoid_rate_avg'] = (ann[['avoid_rate_1','avoid_rate_2','avoid_rate_3']]).mean(axis=1)

In [ ]:
# Determine type of avoidance based on majority choice
def conditions(s):
    flight = (s == 'Flight').sum()
    fight = (s == 'Fight').sum()
    
    if flight > fight:
        return 'Flight'
    elif flight < fight:
        return 'Fight'
    # case when flight=fight
    elif flight > 0:
        return 'Undetermined'
    else:
        return 'NaN'


ann['avoid_type_avg'] = ann[['avoid_type_1','avoid_type_2','avoid_type_3']].apply(conditions, axis=1)

## Statistics

There are 424 QA-pairs in total.

In [ ]:
# One of the annotators marked it as a non-question
ann[(ann[['if_q_1','if_q_2','if_q_3']] == 'NQ').any(axis=1)].shape[0]

83

In [ ]:
print("Rather avoidance:")
print(ann[ann['avoid_rate_avg'] > 2.0].shape[0])
print("Rather non-avoidance:")
print(ann[ann['avoid_rate_avg'] < 2.0].shape[0])
print("Undetermined:")
print(ann[ann['avoid_rate_avg'] == 2.0].shape[0])

Rather avoidance:
174
Rather non-avoidance:
228
Undetermined:
22


In [ ]:
# Tails
print("Definitely avoidance:")
print(ann[ann['avoid_rate_avg'] > 3.5].shape[0])
print("Definitely non-avoidance:")
print(ann[ann['avoid_rate_avg'] < 0.5].shape[0])

Definitely avoidance:
55
Definitely non-avoidance:
135


In [ ]:
def type_stats(df):
    print("Fight cases:")
    print(df[df['avoid_type_avg'] == 'Fight'].shape[0])
    print("Flight cases:")
    print(df[df['avoid_type_avg'] == 'Flight'].shape[0])
    print("Undetermined cases:")
    print(df[df['avoid_type_avg'] == 'Undetermined'].shape[0])
    print("Not rated by any annotator:")
    print(df[df['avoid_type_avg'] == 'NaN'].shape[0])
    
type_stats(ann)

Fight cases:
30
Flight cases:
204
Undetermined cases:
23
Not rated by any annotator:
167


In [ ]:
# For rather avoidance subset
ann_avoided = ann[ann['avoid_rate_avg'] > 2.0]
type_stats(ann_avoided)

Fight cases:
21
Flight cases:
131
Undetermined cases:
22
Not rated by any annotator:
0


In [ ]:
# One of the annotators marked it as a non-question
ann_nq = ann_avoided[(ann_avoided[['if_q_1','if_q_2','if_q_3']] == 'NQ').any(axis=1)]
print(ann_nq.shape[0])
print()
type_stats(ann_nq)

51

Fight cases:
5
Flight cases:
38
Undetermined cases:
8
Not rated by any annotator:
0


## Create files

In [ ]:
ann.to_csv('Avoidance_annotated.csv', index=False)

ann_avoided.to_csv('Avoidance_annotated_avoided.csv', index=False)

ann_avoided[ann_avoided['avoid_type_avg'] == 'Fight'].to_csv('Avoidance_annotated_avoided_fight.csv', index=False)
ann_avoided[ann_avoided['avoid_type_avg'] == 'Flight'].to_csv('Avoidance_annotated_avoided_flight.csv', index=False)